In [311]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ranked-board-game-data-from-boardgamegeek/mechanisms_2023.csv
/kaggle/input/ranked-board-game-data-from-boardgamegeek/reimplementations_2023.csv
/kaggle/input/ranked-board-game-data-from-boardgamegeek/themes_2023.csv
/kaggle/input/ranked-board-game-data-from-boardgamegeek/subdomains_2023.csv
/kaggle/input/ranked-board-game-data-from-boardgamegeek/basic_data_2023.csv


In [312]:
# emojis
!pip install emoji --upgrade
import emoji

In [313]:
# similar names
from fuzzywuzzy import process


In [314]:
# randomize akinator questions
from random import randint

# Reading datasets

In [315]:
df_games = pd.read_csv("/kaggle/input/ranked-board-game-data-from-boardgamegeek/basic_data_2023.csv", 
                       index_col='game_id')
upper_limit_id_games = df_games.index.max()
min_limit_id_games = df_games.index.min()
df_games.columns

Index(['rank', 'bgg_url', 'name', 'min_players', 'max_players', 'avg_time',
       'min_time', 'max_time', 'weight', 'year', 'age', 'avg_rating',
       'geek_rating', 'num_votes', 'owned', 'designer'],
      dtype='object')

In [316]:
df_mechs = pd.read_csv("/kaggle/input/ranked-board-game-data-from-boardgamegeek/mechanisms_2023.csv",
                      index_col = 'game_id')
df_mechs.columns

Index(['Acting', 'Action Drafting', 'Action Points', 'Action Queue',
       'Action Retrieval', 'Action Timer', 'Action/Event', 'Advantage Token',
       'Alliances', 'Area Majority / Influence',
       ...
       'Turn Order: Time Track', 'Variable Phase Order',
       'Variable Player Powers', 'Variable Set-up',
       'Victory Points as a Resource', 'Voting', 'Worker Placement',
       'Worker Placement with Dice Workers',
       'Worker Placement, Different Worker Types', 'Zone of Control'],
      dtype='object', length=188)

In [317]:
df_themes = pd.read_csv('/kaggle/input/ranked-board-game-data-from-boardgamegeek/themes_2023.csv',
                       index_col = 'game_id')


In [318]:
df_subdomains = pd.read_csv("/kaggle/input/ranked-board-game-data-from-boardgamegeek/subdomains_2023.csv",
                           index_col = 'game_id')


# Choose an function option

In [319]:
#initial menu
option = 3 # when finish, change to None
wanted_id = None
while True:
    break #clear after finish
    try:
        option = int(input("""
        Choose an option to search a game:
            [1] from ID;
            [2] from similar names
            [3] randomly
            [4] from mechanics or themes
            [5] Boardgame Akinator
            [0] None, just exit the program\n"""))
    except ValueError:
        print(f'Please, type a valid option.... \n{"-"*30}')
    
    if option in [0,1,2,3,4,5]:
        break
    else:
        print(f'Please, type an number on menu.')

# [1] Searching from ID

In [320]:
#if def functions, erase this cell
continuing = 'S'

while continuing == 'S' and option == 1:
    
    #Structure to ensure game_id does exist
    wanted_id = upper_limit_id_games+1
    while df_games['name'].get(wanted_id) == None:
        try:
            wanted_id = int(input(f"Digit a game ID between {min_limit_id_games} and {upper_limit_id_games}): "))
        except ValueError:
            print(f'Pleade, type a valid number.... \n{"-"*30}')
        while (wanted_id > upper_limit_id_games or wanted_id < min_limit_id_games):
            wanted_id = int(input(f"Digit a game ID between {min_limit_id_games} and {upper_limit_id_games}): "))
        wanted_name = df_games['name'].get(wanted_id)
        if wanted_name == None:
            print("This code doesn't match to any game...")
            wanted_sup = wanted_id
            wanted_inf = wanted_id
            while df_games['name'].get(wanted_sup) == None:
                wanted_sup = wanted_sup + 1

            while df_games['name'].get(wanted_inf) == None:
                wanted_inf = wanted_inf - 1


            print(f"The nearer numbers are {wanted_inf} or {wanted_sup}") 
        else:

            break

    print(f"The game with the ID {wanted_id} is the {wanted_name}.")

   
    continuing = str.upper(input("Gostaria de selecionar outro? [S/N]"))
    if continuing == 'N':
        break

# [2] Searching from similar names

In [321]:
similar_game = 0
while similar_game == 0 and option == 2:
    game = input("Which game are you looking for?")
    all_games = df_games['name']

#compare names
    results = process.extract(game,all_games)
    print(f"the names similars to {game} are: ")
    for similar_game in range(len(results)):
        print(f' [{similar_game+1}] - {results[similar_game][0]}')
    similar_game = int(input('Choose an option: "0" to type again'))
    wanted_id = int((df_games.loc[df_games.name == results[similar_game-1][0]].index)[0])


# [3] Random Game

In [322]:
# call a random valid name
while option == 3:
    wanted_id = randint(min_limit_id_games, upper_limit_id_games+1)
    if df_games.name.get(wanted_id) != None: # não erro
        break
print (wanted_id, df_games.name.get(wanted_id))     

363247 Azul: Master Chocolatier


# [5] Teste Akinator

In [323]:
# comparison formule
def total_players(valor):
    #merging tables
    print(df_games.shape)
    df_intermediate = df_games.merge(df_themes, how = 'left', on = df_games.index)
    df_total = df_intermediate.merge(df_mechs, how = 'left')
    df_total.shape
    # Substitua esta função pela sua lógica de comparação
    if df_total.min_players <= players and df_total.max_players >= players:
            df_total['akinator'] = df_total['akinator'].apply(lambda x: x + 10)

    return valor > 10

# Aplica a fórmula à coluna 'akinator'


In [324]:
df_total['akinator'] = 0
# condition to continue the questions
while df_total['akinator'].nunique() != 1 or df_total['akinator'].max() != df_total['akinator'].iloc[0] and option == 5:
    # Realize as perguntas aqui, atualizando o DataFrame df_total conforme necessário
    players = input("How many players are you looking for?")
        
    # Pode ser uma entrada do usuário ou outro método de atualização
    df_total['comparacao'] = df_total['akinator'].apply(total_players)

    # Atualize os valores na coluna 'akinator' para simular uma mudança nos valores

# Imprima o DataFrame quando a condição for atendida
#print("DataFrame final:")
#print(df_total)

# Printing results

In [325]:
# MECHS/THEMES/SUBDOMAINS: Select the row from the index
wanted_mechs = df_mechs.loc[wanted_id]
wanted_themes = df_themes.loc[wanted_id]
wanted_subdomains = df_subdomains.loc[wanted_id]

# MECHS/THEMES/SUBDOMAINS: Obtain the nemas of columns whete from the index
wanted_mechs = wanted_mechs.index[wanted_mechs == 1].tolist()
wanted_themes = wanted_themes.index[wanted_themes == 1].tolist()
wanted_subdomains = wanted_subdomains.index[wanted_subdomains == 1].tolist()

# MECHS
df_similar_mechs = df_mechs[wanted_mechs]
df_similar_mechs['sum'] = df_similar_mechs.sum(axis=1)

# MECHS: list of 2 top sums
top2_sums_mechs = df_similar_mechs['sum'].unique().tolist()
top2_sums_mechs.sort(reverse=True)
top2_sums_mechs = top2_sums_mechs[0:2]

# MECHS: exclude 0'es in list 
if 0 in top2_sums_mechs:
    top2_sums_mechs.remove(0)

# MECHS: games with 2 top sums    
only_most_similar_games_mechs = df_similar_mechs.loc[df_similar_mechs['sum'].isin(top2_sums_mechs)].index.tolist()

# MECHS: Remove elements equal to search item
only_most_similar_games_mechs.remove(wanted_id)
only_most_similar_games_mechs = df_games.name.loc[only_most_similar_games_mechs].tolist()

# THEMES
df_similar_themes = df_themes[wanted_themes]
df_similar_themes['sum'] = df_similar_themes.sum(axis=1)


# THEMES: list of 2 top sums
top2_sums_themes = df_similar_themes['sum'].unique().tolist()
top2_sums_themes.sort(reverse=True)
top2_sums_themes = top2_sums_themes[0:2]

# THEMES: exclude 0'es in list 
if 0 in top2_sums_themes:
    top2_sums_themes.remove(0)

# THEMES: games with 2 top sums    
only_most_similar_games_themes = df_similar_themes.loc[df_similar_themes['sum'].isin(top2_sums_themes)].index.tolist()

#  THEMES: Remove os elementos iguais ao valor especificado
only_most_similar_games_themes.remove(wanted_id)
only_most_similar_games_themes = df_games.name.loc[only_most_similar_games_themes].tolist()


/tmp/ipykernel_42/2849690178.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_similar_mechs['sum'] = df_similar_mechs.sum(axis=1)
/tmp/ipykernel_42/2849690178.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_similar_themes['sum'] = df_similar_themes.sum(axis=1)


In [326]:
input("Press ENTER to see the results")
print(15*'-*-')
print(emoji.emojize(f":bullseye: The game with the code {wanted_id} is {df_games['name'].get(wanted_id)}, link {df_games.bgg_url[wanted_id]}. \n"))
print(15*'-*-')

print(emoji.emojize("📗"))
print(df_games.loc[wanted_id,['rank','min_players', 'max_players', 'avg_time','min_time', 
'max_time', 'weight', 'year', 'age', 'avg_rating','geek_rating', 'num_votes', 
'owned', 'designer']])

print(15*'-*-')

print(f"The subdomains are: {wanted_subdomains}")
print(15*'-*-')

print(emoji.emojize(f"🛠 MECHANICS \nThe mechanics in this game are:"))
for item in range(len(wanted_mechs)):
    print(f"  - {wanted_mechs[item]}")
    
print(f'The games with similar mechanics are: ')
for item in range(len(only_most_similar_games_mechs)):
    print(f'  - {only_most_similar_games_mechs[item]} - year ', end = "")
    print(df_games.loc[df_games.name == only_most_similar_games_mechs[item],'year'].iloc[0], end = "")
    print(" - weight (0 to 5) ", end = "")
    print(df_games.loc[df_games['name'] == only_most_similar_games_mechs[item],'weight'].iloc[0])
print(15*'-*-')

print(emoji.emojize(f"\n 📜 THEMES \nThe themes in this game are:"))
for item in range(len(wanted_themes)):
    print(f"  - {wanted_themes[item]}") 
print(f'The games with similar themes are: ')
for item in range(len(only_most_similar_games_themes)):
    print(f'  - {only_most_similar_games_themes[item]} - ano ', end = "")
    print(df_games.loc[df_games.name == only_most_similar_games_themes[item],'year'].iloc[0], end = "")
    print(" - peso ", end = "")
    print(df_games.loc[df_games['name'] == only_most_similar_games_themes[item],'weight'].iloc[0])



Press ENTER to see the results 


-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
🎯 The game with the code 363247 is Azul: Master Chocolatier, link https://boardgamegeek.com/boardgame/363247. 

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
📗
rank                       1764
min_players                   2
max_players                   4
avg_time                     45
min_time                     30
max_time                     45
weight                   1.7083
year                       2022
age                           8
avg_rating              7.95389
geek_rating             6.25161
num_votes                   900
owned                      2739
designer       Michael Kiesling
Name: 363247, dtype: object
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
The subdomains are: ['Abstract Games']
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
🛠 MECHANICS 
The mechanics in this game are:
  - End Game Bonuses
  - Open Drafting
  - Pattern Building
  - Tile Placement
  - Turn Order: Claim Action
The games with similar mechanics a